In [20]:
import sqlite3
from sqlite3 import Error
from typing import Tuple, List, Union
import random
from random import choice
import csv

In [21]:
def read_data_from_csv(csv_path:str)->Tuple:
    """
    read data from a csv file (no input validataion)
    @param csv_path: path to csv file
    @return: header, and data in the form of list of tuple
    """
    with open(csv_path) as file:
        csvreader = csv.reader(file)
        header = next(csvreader)
        print(header)
        rows = []
        for row in csvreader:
            rows.append(tuple(row))
        return header, rows
    
    
    

In [22]:
def create_database(db_file:str, schema_file: str):
    """ 
    create a database connection to a SQLite database and create tables based on schema file 
    @param db_file: path to db file
    @param schema_file: path to schema file
    """
    connection = None
    try:
        connection = sqlite3.connect(db_file)
        with open(schema_file) as file:
            connection.executescript(file.read())
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if connection:
            connection.close()

In [23]:
def insert_entry_to_table(db_file:str, entry_value: Tuple, table_name: str,key_name: Union[List[str],Tuple]):
    """ 
    insert one entry to the demo table 
    @param db_file: path to db file
    @param entry_value: value of entry to be inserted
    @param key_name: name of each key in the database
    @param table_name: name of table
    """
    connection = None
    try:
        assert len(key_name) == len(entry_value)
        connection = sqlite3.connect(db_file)
        cur = connection.cursor()
        key_name_str = str(tuple(key_name))
        value_str = "(" + " ?,"*(len(key_name)-1) + " ?)"
        query = f"INSERT INTO {table_name} {key_name_str} VALUES {value_str}"
        cur.execute(query, entry_value)
    except Error as e:
        print(e)
    finally:
        if connection:
            connection.commit()
            connection.close()
    

In [24]:
# def insert_entry_demo_table_with_default_time(db_file:str, entry_value: Tuple):
#     """ 
#     insert one entry to the demo table 
#     @param db_file: path to db file
#     @param entry_value: value of entry to be inserted
#     @param entry_name: name of each key in the database
#     """
#     connection = None
#     try:
#         connection = sqlite3.connect(db_file)
#         cur = connection.cursor()
#         query = "INSERT INTO demo_table (member_name, company, value) VALUES (?, ?, ?)"
#         cur.execute(query, entry_value)
#     except Error as e:
#         print(e)
#     finally:
#         if connection:
#             connection.commit()
#             connection.close()
    

In [25]:
        
def insert_entries_to_table(db_file:str, entries: List, table_name:str, key_name: Union[List[str],Tuple[str]]):
    """ 
    insert entries to the demo table 
    @param db_file: path to db file
    @param entries: list of tuples of values of entries to be inserted
    @param key_name: name of each key in the database
    @param table_name: name of table
    """
    connection = None
    try:
        connection = sqlite3.connect(db_file)
        cur = connection.cursor()
        key_name_str = str(tuple(key_name))
        value_str = "(" + " ?,"*(len(key_name)-1) + " ?)"
        query = f"INSERT INTO {table_name} {key_name_str} VALUES {value_str}"
        cur.executemany(query, entries)
    except Error as e:
        print(e)
    finally:
        if connection:
            connection.commit()
            connection.close()
    

In [26]:
# def insert_entries_demo_table_with_default_time(db_file:str, entries: List):
#     """ 
#     insert entries to the demo table 
#     @param db_file: path to db file
#     @param entries: list of tuples of values of entries to be inserted
#     """
#     connection = None
#     try:
#         connection = sqlite3.connect(db_file)
#         cur = connection.cursor()
#         query = "INSERT INTO demo_table (member_name, company, value) VALUES (?, ?, ?)"
#         cur.executemany(query, entries)
#     except Error as e:
#         print(e)
#     finally:
#         if connection:
#             connection.commit()
#             connection.close()
    

In [27]:
def get_all_entries_from_db(db_file: str, table_name: str) -> List[str]:
    """ 
    insert entries to the demo table 
    @param db_file: path to db file
    @param table_name: name of the table that we are interested in
    @return: a list of all entries
    """
    connection = None
    try:
        connection = sqlite3.connect(db_file)
        cur = connection.cursor()
        query = f'SELECT * FROM {table_name} ORDER BY transaction_date'
        entries = cur.execute(query)
        for row in entries:
            print(row)
    except Error as e:
        print(e)
    finally:
        if connection:
            connection.close()
            return entries
    

In [28]:
#parameters we are going to use
data_base_name = r"v1.db"
sql_schema = r"v1_schema.sql"
table_name = r"all_transaction"
csv_path = r"v1_data.csv"

In [29]:
# parse data
header, data = read_data_from_csv(csv_path)

['member_name', 'state_district_number', 'company', 'ticker', 'transaction_type', 'transaction_date', 'value_lb', 'value_ub', 'description', 'link']


In [30]:
# create database
create_database(data_base_name, schema_file=sql_schema)

2.6.0


In [31]:
# test insert one entry
insert_entry_to_table(db_file=data_base_name, entry_value=data[0],table_name=table_name, key_name=header)

In [32]:
entries = get_all_entries_from_db(db_file=data_base_name, table_name=table_name)

(1, 'Nancy Pelosi', 'CA12', 'Tesla, Inc', 'TSLA', 'P', '2022-03-17', 1000001, 5000000, 'Exercised 25 call options(2,500 shares) expiring 3/18/22 at a strike price of $500.', 'https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/2022/20020662.pdf')


In [33]:
# test insert many
insert_entries_to_table(db_file=data_base_name, entries=data[1:],table_name=table_name, key_name=header)

In [34]:
entries = get_all_entries_from_db(db_file=data_base_name, table_name=table_name)

(2, 'Robert E. Andrews', 'NJ01', 'Hill International, Inc', 'HIL', 'S', '2013-12-26', 15001, 50000, 'None', 'https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/2014/20000077.pdf')
(7, 'Eric Cantor', 'VA07', 'Mesabi Trust', 'MSB', 'P', '2014-01-16', 15001, 50000, 'None', 'https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/2014/20000165.pdf')
(4, 'John A. Boehner', 'OH08', 'Diamond Offshore Drilling, Inc', 'DO', 'S', '2014-02-06', 1001, 15000, 'None', 'https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/2014/20000284.pdf')
(6, 'Michael C. Burgess', 'TX26', 'General Electric Company', 'GE', 'S', '2014-03-10', 1001, 15000, 'None', 'https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/2014/20000708.pdf')
(5, 'Mo Brooks', 'AL05', 'Morgan Stanley Cap Tr V Gtd Secs', 'MWO', 'S', '2014-05-05', 1001, 15000, 'None', 'https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/2014/20000606.pdf')
(3, 'Lou Barletta', 'PA11', 'Gigamon Inc', 'GIMO', 'S', '2014-05-22', 1001, 15000, '

In [48]:
connection = sqlite3.connect(data_base_name)
cur = connection.cursor()
query = f'SELECT * from {table_name} where strftime(\'%Y\',transaction_date) = \'2014\''
q= r"SELECT strftime('%Y','2022-04-10')"
entries = cur.execute(query)
for row in entries:
    print(row)

(3, 'Lou Barletta', 'PA11', 'Gigamon Inc', 'GIMO', 'S', '2014-05-22', 1001, 15000, 'None', 'https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/2014/20000998.pdf')
(4, 'John A. Boehner', 'OH08', 'Diamond Offshore Drilling, Inc', 'DO', 'S', '2014-02-06', 1001, 15000, 'None', 'https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/2014/20000284.pdf')
(5, 'Mo Brooks', 'AL05', 'Morgan Stanley Cap Tr V Gtd Secs', 'MWO', 'S', '2014-05-05', 1001, 15000, 'None', 'https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/2014/20000606.pdf')
(6, 'Michael C. Burgess', 'TX26', 'General Electric Company', 'GE', 'S', '2014-03-10', 1001, 15000, 'None', 'https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/2014/20000708.pdf')
(7, 'Eric Cantor', 'VA07', 'Mesabi Trust', 'MSB', 'P', '2014-01-16', 15001, 50000, 'None', 'https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/2014/20000165.pdf')
(10, 'William M. Cassidy', 'LA06', 'Chevron Corporation', 'CVX', 'P', '2014-12-16', 15001, 50000, 'No